In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [2]:
data = pd.read_csv('/Users/soroushesnaashari/Documents/GitHub/Repo 4. Market-Analyzing/Market.csv', encoding='unicode_escape')
market_data = data.copy()
market_data

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,12/31/1965,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1/3/1966,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1/4/1966,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1/5/1966,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1/6/1966,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
112452,N100,5/27/2021,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0
112453,N100,5/28/2021,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0
112454,N100,5/31/2021,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0
112455,N100,6/1/2021,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0


# Preparing the Dataset

In [3]:
# Filter rows where Index is "NYA"
nya_data = market_data[market_data['Index'] == "NYA"].copy()
# Remove the decimals
nya_data = round(nya_data)
# Drop the "Index" and "Volume" columns
nya_data = nya_data.drop(columns = ["Index", "Volume"])
# Removing the missing values
nya_data = nya_data.dropna()
nya_data

,Date,Open,High,Low,Close,Adj Close
0,12/31/1965,529.0,529.0,529.0,529.0,529.0
1,1/3/1966,527.0,527.0,527.0,527.0,527.0
2,1/4/1966,528.0,528.0,528.0,528.0,528.0
3,1/5/1966,531.0,531.0,531.0,531.0,531.0
4,1/6/1966,532.0,532.0,532.0,532.0,532.0
...,...,...,...,...,...,...
13943,5/24/2021,16375.0,16509.0,16375.0,16465.0,16465.0
13944,5/25/2021,16465.0,16526.0,16375.0,16390.0,16390.0
13945,5/26/2021,16390.0,16466.0,16388.0,16452.0,16452.0
13946,5/27/2021,16452.0,16546.0,16452.0,16532.0,16532.0


In [4]:
# Finding the first noise
first_noise = nya_data[(nya_data["Open"] > 5000) & (nya_data["Adj Close"] < 1000)]
first_noise

,Date,Open,High,Low,Close,Adj Close
852,6/30/1969,5722.0,572.0,572.0,572.0,572.0


In [5]:
# Finding the second noise
second_noise = nya_data[(nya_data["Open"] < 1000) & (nya_data["Adj Close"] > 6000)]
second_noise

,Date,Open,High,Low,Close,Adj Close
831,5/29/1969,612.0,612.0,612.0,612.0,6111.0


In [6]:
new_nya_data = nya_data.drop(index = [852, 831])
new_nya_data

,Date,Open,High,Low,Close,Adj Close
0,12/31/1965,529.0,529.0,529.0,529.0,529.0
1,1/3/1966,527.0,527.0,527.0,527.0,527.0
2,1/4/1966,528.0,528.0,528.0,528.0,528.0
3,1/5/1966,531.0,531.0,531.0,531.0,531.0
4,1/6/1966,532.0,532.0,532.0,532.0,532.0
...,...,...,...,...,...,...
13943,5/24/2021,16375.0,16509.0,16375.0,16465.0,16465.0
13944,5/25/2021,16465.0,16526.0,16375.0,16390.0,16390.0
13945,5/26/2021,16390.0,16466.0,16388.0,16452.0,16452.0
13946,5/27/2021,16452.0,16546.0,16452.0,16532.0,16532.0


In [7]:
Third_noise = new_nya_data[(new_nya_data["High"] > 6000) & (new_nya_data["Adj Close"] < 1000)]
Third_noise

,Date,Open,High,Low,Close,Adj Close
829,5/27/1969,612.0,6124.0,612.0,612.0,612.0
833,6/3/1969,607.0,6066.0,607.0,607.0,607.0


In [8]:
new1_nya_data = new_nya_data.drop(index = [829, 833])
new1_nya_data

,Date,Open,High,Low,Close,Adj Close
0,12/31/1965,529.0,529.0,529.0,529.0,529.0
1,1/3/1966,527.0,527.0,527.0,527.0,527.0
2,1/4/1966,528.0,528.0,528.0,528.0,528.0
3,1/5/1966,531.0,531.0,531.0,531.0,531.0
4,1/6/1966,532.0,532.0,532.0,532.0,532.0
...,...,...,...,...,...,...
13943,5/24/2021,16375.0,16509.0,16375.0,16465.0,16465.0
13944,5/25/2021,16465.0,16526.0,16375.0,16390.0,16390.0
13945,5/26/2021,16390.0,16466.0,16388.0,16452.0,16452.0
13946,5/27/2021,16452.0,16546.0,16452.0,16532.0,16532.0


In [11]:
features = ['Open', 'High', 'Low', 'Adj Close']
target = 'Close'

X = new1_nya_data[features]
y = new1_nya_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

To find out more about choosing between "StandardScaler" or "MinMaxScaler" you can read the article below: https://medium.com/@soroushesnaashari/data-scaling-101-standardscaler-vs-minmaxscaler-in-machine-learning-ff88b7810a48

---

# Random Forest

### Creating the Model

In [13]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)

grid_search.fit(X_train, y_train)
best_params_rf = grid_search.best_params_
best_rf = grid_search.best_estimator_

y_pred_rf_tuned = best_rf.predict(X_test)
mse_rf_tuned = mean_squared_error(y_test, y_pred_rf_tuned)

print("Best Parameters:", best_params_rf)
print("Tuned Random Forest MSE:", mse_rf_tuned)

Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.6s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; tot

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   2.8s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   3.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   2.5s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   1.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   2.0s
[CV] END bootstrap=True, max_depth=10, max_featu

[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   1.8s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   1.7s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   2.6s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   2.6s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   2.5s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.8s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.8s
[CV] END bootstrap=True, max_depth=10, max_fea

[CV] END bootstrap=True, max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   1.3s
[CV] END bootstrap=True, max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   2.7s
[CV] END bootstrap=True, max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   2.7s
[CV] END bootstrap=True, max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   2.7s
[CV] END bootstrap=True, max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   4.0s
[CV] END bootstrap=True, max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   4.1s
[CV] END bootstrap=True, max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   4.0s
[CV] END bootstrap=True, max_depth=10, ma

[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.1s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.1s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.1s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   3.2s
[CV] END bootstrap=True, max_depth=20, max_featu

...

[CV] END bootstrap=False, max_depth=30, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   6.3s
[CV] END bootstrap=False, max_depth=30, max_features=None, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.0s
[CV] END bootstrap=False, max_depth=30, max_features=None, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.1s
[CV] END bootstrap=False, max_depth=30, max_features=None, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.0s
[CV] END bootstrap=False, max_depth=30, max_features=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   4.3s
[CV] END bootstrap=False, max_depth=30, max_features=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   4.1s
[CV] END bootstrap=False, max_depth=30, max_features=None, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   4.1s
[CV] END bootstrap=False, max_depth

[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.4s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   2.3s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   3.5s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   3.6s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   3.6s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END bootstrap=Fal

[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.4s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.4s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   2.7s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   2.7s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   2.6s
[CV] END bootstrap=False, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   4.4s
[CV] END bootstrap=False, m

[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   5.2s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   7.9s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   7.9s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   7.9s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.4s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.4s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.3s
[CV] END bootstrap=False, m

[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   2.0s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   2.0s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   3.9s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   3.9s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   3.9s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   6.0s
[CV] END bootstrap=False, max_depth=None, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   6.3s
[CV] END bootstrap=False, m

### Results

I trained the model with **"Random Forest"**, it takes about **two hours** to run the GridSearch and the most optimized version is:

Best Parameters:<br>

'bootstrap': **False**<br>
'max_depth': **20**<br>
'max_features': **None**<br>
'min_samples_leaf': **1**<br>
'min_samples_split': **2**<br>
'n_estimators': **200**<br>

Tuned Random Forest MSE: **10.316188585786048**

---

# Decision Tree

### Creating the Model

In [17]:
param_dist = {
    'max_depth': [None] + list(np.arange(1, 50)),
    'min_samples_split': np.arange(2, 20),
    'min_samples_leaf': np.arange(1, 20),
    'max_features': [None, 'sqrt', 'log2'],
    'splitter': ['best', 'random']
}

dt = DecisionTreeRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_dist,
    n_iter=300,    # Number of iteration
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=2
)

random_search.fit(X_train, y_train)
best_params_dt = random_search.best_params_
best_dt = random_search.best_estimator_

y_pred_dt_tuned = best_dt.predict(X_test)
mse_dt_tuned = mean_squared_error(y_test, y_pred_dt_tuned)

print("Best Parameters for Decision Tree:", best_params_dt)
print("Tuned Decision Tree MSE:", mse_dt_tuned)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
[CV] END max_depth=7, max_features=log2, min_samples_leaf=2, min_samples_split=15, splitter=random; total time=   0.0s
[CV] END max_depth=7, max_features=log2, min_samples_leaf=2, min_samples_split=15, splitter=random; total time=   0.0s
[CV] END max_depth=7, max_features=log2, min_samples_leaf=2, min_samples_split=15, splitter=random; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=5, min_samples_split=18, splitter=best; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=5, min_samples_split=18, splitter=best; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=5, min_samples_split=18, splitter=best; total time=   0.0s
[CV] END max_depth=37, max_features=sqrt, min_samples_leaf=6, min_samples_split=18, splitter=best; total time=   0.0s
[CV] END max_depth=37, max_features=sqrt, min_samples_leaf=6, min_samples_split=18, splitter=best; tot

[CV] END max_depth=41, max_features=None, min_samples_leaf=10, min_samples_split=13, splitter=best; total time=   0.0s
[CV] END max_depth=41, max_features=None, min_samples_leaf=10, min_samples_split=13, splitter=best; total time=   0.0s
[CV] END max_depth=41, max_features=None, min_samples_leaf=10, min_samples_split=13, splitter=best; total time=   0.0s
[CV] END max_depth=8, max_features=log2, min_samples_leaf=18, min_samples_split=19, splitter=random; total time=   0.0s
[CV] END max_depth=8, max_features=log2, min_samples_leaf=18, min_samples_split=19, splitter=random; total time=   0.0s
[CV] END max_depth=8, max_features=log2, min_samples_leaf=18, min_samples_split=19, splitter=random; total time=   0.0s
[CV] END max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, splitter=random; total time=   0.0s
[CV] END max_depth=1, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, splitter=random; total time=   0.0s
[CV] END max_depth=1, max_features=sqrt, min_sa

[CV] END max_depth=39, max_features=None, min_samples_leaf=2, min_samples_split=8, splitter=random; total time=   0.0s
[CV] END max_depth=39, max_features=None, min_samples_leaf=2, min_samples_split=8, splitter=random; total time=   0.0s
[CV] END max_depth=39, max_features=None, min_samples_leaf=2, min_samples_split=8, splitter=random; total time=   0.0s
[CV] END max_depth=17, max_features=sqrt, min_samples_leaf=10, min_samples_split=16, splitter=best; total time=   0.0s
[CV] END max_depth=17, max_features=sqrt, min_samples_leaf=10, min_samples_split=16, splitter=best; total time=   0.0s
[CV] END max_depth=17, max_features=sqrt, min_samples_leaf=10, min_samples_split=16, splitter=best; total time=   0.0s
[CV] END max_depth=44, max_features=log2, min_samples_leaf=12, min_samples_split=9, splitter=random; total time=   0.0s
[CV] END max_depth=44, max_features=log2, min_samples_leaf=12, min_samples_split=9, splitter=random; total time=   0.0s
[CV] END max_depth=44, max_features=log2, min_

[CV] END max_depth=19, max_features=log2, min_samples_leaf=2, min_samples_split=4, splitter=random; total time=   0.0s
[CV] END max_depth=19, max_features=log2, min_samples_leaf=2, min_samples_split=4, splitter=random; total time=   0.0s
[CV] END max_depth=19, max_features=log2, min_samples_leaf=2, min_samples_split=4, splitter=random; total time=   0.0s
[CV] END max_depth=44, max_features=sqrt, min_samples_leaf=12, min_samples_split=11, splitter=random; total time=   0.0s
[CV] END max_depth=44, max_features=sqrt, min_samples_leaf=12, min_samples_split=11, splitter=random; total time=   0.0s
[CV] END max_depth=44, max_features=sqrt, min_samples_leaf=12, min_samples_split=11, splitter=random; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=10, min_samples_split=6, splitter=best; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=10, min_samples_split=6, splitter=best; total time=   0.0s
[CV] END max_depth=None, max_features=sq

[CV] END max_depth=19, max_features=None, min_samples_leaf=15, min_samples_split=8, splitter=best; total time=   0.0s
[CV] END max_depth=19, max_features=None, min_samples_leaf=15, min_samples_split=8, splitter=best; total time=   0.0s
[CV] END max_depth=16, max_features=None, min_samples_leaf=10, min_samples_split=3, splitter=random; total time=   0.0s
[CV] END max_depth=16, max_features=None, min_samples_leaf=10, min_samples_split=3, splitter=random; total time=   0.0s
[CV] END max_depth=16, max_features=None, min_samples_leaf=10, min_samples_split=3, splitter=random; total time=   0.0s
[CV] END max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=11, splitter=best; total time=   0.0s
[CV] END max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=11, splitter=best; total time=   0.0s
[CV] END max_depth=6, max_features=log2, min_samples_leaf=9, min_samples_split=11, splitter=best; total time=   0.0s
[CV] END max_depth=30, max_features=None, min_samples

...

[CV] END max_depth=47, max_features=sqrt, min_samples_leaf=11, min_samples_split=10, splitter=best; total time=   0.0s
[CV] END max_depth=47, max_features=sqrt, min_samples_leaf=11, min_samples_split=10, splitter=best; total time=   0.0s
[CV] END max_depth=47, max_features=log2, min_samples_leaf=8, min_samples_split=19, splitter=best; total time=   0.0s
[CV] END max_depth=47, max_features=log2, min_samples_leaf=8, min_samples_split=19, splitter=best; total time=   0.0s
[CV] END max_depth=47, max_features=log2, min_samples_leaf=8, min_samples_split=19, splitter=best; total time=   0.0s
[CV] END max_depth=6, max_features=sqrt, min_samples_leaf=16, min_samples_split=6, splitter=random; total time=   0.0s
[CV] END max_depth=6, max_features=sqrt, min_samples_leaf=16, min_samples_split=6, splitter=random; total time=   0.0s
[CV] END max_depth=6, max_features=sqrt, min_samples_leaf=16, min_samples_split=6, splitter=random; total time=   0.0s
[CV] END max_depth=32, max_features=None, min_sampl

[CV] END max_depth=13, max_features=None, min_samples_leaf=2, min_samples_split=14, splitter=best; total time=   0.0s
[CV] END max_depth=13, max_features=None, min_samples_leaf=2, min_samples_split=14, splitter=best; total time=   0.0s
[CV] END max_depth=45, max_features=log2, min_samples_leaf=14, min_samples_split=18, splitter=random; total time=   0.0s
[CV] END max_depth=45, max_features=log2, min_samples_leaf=14, min_samples_split=18, splitter=random; total time=   0.0s
[CV] END max_depth=45, max_features=log2, min_samples_leaf=14, min_samples_split=18, splitter=random; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=5, min_samples_split=15, splitter=best; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=5, min_samples_split=15, splitter=best; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=5, min_samples_split=15, splitter=best; total time=   0.0s
[CV] END max_depth=43, max_features=log2,

[CV] END max_depth=42, max_features=None, min_samples_leaf=1, min_samples_split=11, splitter=best; total time=   0.0s
[CV] END max_depth=42, max_features=None, min_samples_leaf=1, min_samples_split=11, splitter=best; total time=   0.0s
[CV] END max_depth=25, max_features=None, min_samples_leaf=11, min_samples_split=3, splitter=random; total time=   0.0s
[CV] END max_depth=25, max_features=None, min_samples_leaf=11, min_samples_split=3, splitter=random; total time=   0.0s
[CV] END max_depth=25, max_features=None, min_samples_leaf=11, min_samples_split=3, splitter=random; total time=   0.0s
[CV] END max_depth=7, max_features=sqrt, min_samples_leaf=19, min_samples_split=8, splitter=best; total time=   0.0s
[CV] END max_depth=7, max_features=sqrt, min_samples_leaf=19, min_samples_split=8, splitter=best; total time=   0.0s
[CV] END max_depth=7, max_features=sqrt, min_samples_leaf=19, min_samples_split=8, splitter=best; total time=   0.0s
[CV] END max_depth=44, max_features=log2, min_samples

[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=17, splitter=best; total time=   0.0s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=17, splitter=best; total time=   0.0s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=17, splitter=best; total time=   0.0s
[CV] END max_depth=41, max_features=log2, min_samples_leaf=14, min_samples_split=8, splitter=random; total time=   0.0s
[CV] END max_depth=41, max_features=log2, min_samples_leaf=14, min_samples_split=8, splitter=random; total time=   0.0s
[CV] END max_depth=41, max_features=log2, min_samples_leaf=14, min_samples_split=8, splitter=random; total time=   0.0s
[CV] END max_depth=29, max_features=sqrt, min_samples_leaf=15, min_samples_split=10, splitter=random; total time=   0.0s
[CV] END max_depth=29, max_features=sqrt, min_samples_leaf=15, min_samples_split=10, splitter=random; total time=   0.0s
[CV] END max_depth=29, max_features=sqrt, mi

[CV] END max_depth=28, max_features=log2, min_samples_leaf=7, min_samples_split=2, splitter=best; total time=   0.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=16, min_samples_split=9, splitter=random; total time=   0.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=16, min_samples_split=9, splitter=random; total time=   0.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=16, min_samples_split=9, splitter=random; total time=   0.0s
[CV] END max_depth=14, max_features=None, min_samples_leaf=16, min_samples_split=17, splitter=random; total time=   0.0s
[CV] END max_depth=14, max_features=None, min_samples_leaf=16, min_samples_split=17, splitter=random; total time=   0.0s
[CV] END max_depth=14, max_features=None, min_samples_leaf=16, min_samples_split=17, splitter=random; total time=   0.0s
[CV] END max_depth=6, max_features=None, min_samples_leaf=16, min_samples_split=13, splitter=best; total time=   0.0s
[CV] END max_depth=6, max_features=None, min_

### Results
I used 3 iterations and you can find all the answers below:

Best Parameters for Decision Tree using **100** Iterations: 

'splitter': **'best'**<br>
'min_samples_split': **7**<br>
'min_samples_leaf': **1**<br>
'max_features': **None**<br>
'max_depth': **39**<br>

Tuned Decision Tree MSE: **19.52277947276057**

---

Best Parameters for Decision Tree using **200** iterations:

'splitter': **'best'**<br>
'min_samples_split': **7**<br>
'min_samples_leaf': **1**<br>
'max_features': **None**<br>
'max_depth': **39**<br>

Tuned Decision Tree MSE: **19.52277947276057**

---

Best Parameters for Decision Tree using **300** iterations:

'splitter': **'best'**<br>
'min_samples_split': **7**<br>
'min_samples_leaf': **1**<br>
'max_features': **None**<br>
'max_depth': **39**<br>

Tuned Decision Tree MSE: **19.52277947276057**

Nothing changed in these three iterations so the most optimized answer would be **19.52277947276057** which is not better than random forest's score

# Final Conclusion

The best result that I found either in ver. 0.0 or in ver. 1.0, is **10.316188585786048**, which found by tuning the hyperparameters of **Random Forest** algorithm

# The End !!!
Mohammad Soroush Esnaashari